CódigraphTesto disponível [na minha página do Github](https://github.com/arthurkenzo/atividades_ia525)

### Importando bibliotecas

In [56]:
import cvxpy as cp
import numpy as np
import mosek
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import networkx as nx
import random


from typing import Tuple
from itertools import product


## Questão 1: Pintar o tabuleiro

Para obter o modelo, notamos primeiramente que, como as operações de inversão comutam entre si, o estado final de uma dada casa $(i,j)$ no tabuleiro depende somente do número total de inversões aplicadas em sua vizinhança (contando também a própria casa). Se esse número for ímpar, o estado da casa é invertido, senão seu estado não muda. Para saber o estado final de uma dada casa, precisamos então de uma variável que indique se o total de inversões em uma dada vizinhança é par ou ímpar. 

Isso pode ser feito considerando a representação binária do total de inversões: qualquer binário _par_ tem seu dígito menos significativo igual a $0$, assim como qualquer _ímpar_ tem seu dígito menos significativo igual a $1$. A partir dessa variável indicatriz, podemos definir condições para que uma dada casa $(i,j)$ seja preenchida ao final do jogo: se ela está inicialmente preenchida, precisamos aplicar um total _par_ de inversões em sua vizinhança, caso contrário precisamos de um número _ímpar_ de inversões. Em termos de lógica booleana, temos então a condição $m_{ij}\: \text{XOR}\: z_{ij}^{(0)}$, onde $m_{ij}$ é o valor inicial da casa e $z_{ij}^{(0)}$ é o dígito menos significativo da representação binária do total de inversões aplicadas.

Um jogo é então _factível_ se essa condição é respeitada para todas as casas do tabuleiro. O jogo _ótimo_ será dado pelo jogo factível com menor número de inversões aplicadas no tabuleiro todo. Chegamos então ao seguinte modelo:

### Modelo:

\begin{align*}
    \text{Minimizar }   & x_{ij}\\
    \text{sujeito a }   & \sum_{i'j'\in N(i,j)} x_{ij} = 4z_{ij}^{(4)} + 2z_{ij}^{(2)} + z_{ij}^{(1)} \quad \forall  i,j  \hspace{20pt} \text{(Codificação binária do total de inversões aplicadas)} \\
                        & m_{ij} + z_{ij}^{(0)} = 1 \quad \forall  i,j  \hspace{20pt} \text{(Inversões necessárias para deixar a casa pintada)} \\
\end{align*}

Neste modelo, $x_{ij}$ denota variáveis de decisão tais que $x_{ij} = 1$ $\iff$ uma inversão é aplicada à célula $(i,j)$ e as células adjacentes, $N(i,j)$ denota o conjunto de células afetadas por uma operação de inversão em $(i,j)$, $z_{ij}^{(n)}$ é o $n$-ésimo dígito da codificação binária do número total de inversões na vizinhança de cada célula. O estado inicial do tabuleiro é dado por $m_{ij}$, onde $m(i,j)=1$ $\iff$ a célula está inicialmente preenchida. 

In [57]:
def SolvePintarTabuleiro(board:np.ndarray) -> np.ndarray:

    boardShape = board.shape

    # defining decision and auxiliary variables
    flipVars = cp.Variable(boardShape, boolean=True, name="flip")
    encodingVars1 = cp.Variable(boardShape, boolean=True, name="encode0")
    encodingVars2 = cp.Variable(boardShape, boolean=True, name="encode2")
    encodingVars4 = cp.Variable(boardShape, boolean=True, name="encode4")

    # constraints
    constraints = []

    # encoding the number of incident flips onto binary variables
    for i in range(boardShape[0]):
        for j in range(boardShape[1]):

            expression = flipVars[i, j]  # start with the cell itself
            if i > 0:
                expression += flipVars[i - 1, j]  # up
            if i < boardShape[0] - 1:
                expression += flipVars[i + 1, j]  # down
            if j > 0:
                expression += flipVars[i, j - 1]  # left
            if j < boardShape[1] - 1:
                expression += flipVars[i, j + 1]  # right

            # Example: no more than 2 adjacent ON cells including itself
            constraints.append(expression == 4*encodingVars4[i,j] + 2*encodingVars2[i,j] + encodingVars1[i,j])

    # flip codition: if total number of incident flips is odd, cell is flipped 
    for i in range(boardShape[0]):
        for j in range(boardShape[1]):
            constraints += [board[i,j] + encodingVars1[i,j] == 1]


    # minimizing total number of flips on the board
    objective = cp.Minimize(cp.sum(flipVars))

    # solving
    lp = cp.Problem(objective, constraints)
    lp.solve(solver="MOSEK", verbose=False)

    return flipVars.value.round(1)

In [58]:
def main():
    # dado de entrada - matriz do tabuleiro
    M = np.array([[1,0,1,1,1],[1,0,1,0,1],[0,0,1,0,0],[1,0,0,1,1],[0,1,0,1,0],[1,1,0,1,1]])
    
    ### implementação do CVXPY --- tarefa
    X = SolvePintarTabuleiro(M)
    print("Número de jogadas: ", np.sum(X))

    # resposta obtida pelo CVXPY    
    # X = np.array([[1,0,1,0,0],[1,1,1,1,0],[1,0,0,1,1],[1,1,0,0,1],[1,1,0,0,0],[0,1,0,0,0]])
    # print("Número de jogadas: ", np.sum(X))
    
    # "interface gráfica" para observar as jogadas realizadas
    tabuleiroResolvido = tabuleiro(X,M)

    if np.sum(tabuleiroResolvido) == M.shape[0]*M.shape[1]:
        print("Tabuleiro resolvido com sucesso")
    else:
        print("Tabuleiro não foi resolvido.")


def imprimir(_M,jogada) :
    [m,n] = _M.shape
    #C = colormap([1 1 1; 0 0 0; 0 1 0]
    fig, ax = plt.subplots()
    i = ax.imshow(_M, interpolation='nearest', cmap=cm.BuPu, vmin=0, vmax=2)
    #fig.colorbar(i)
    print("Jogada ",str(jogada)) 
    plt.show()

def inverter(val) :
    if val == 0 :
        return 1
    if val == 1 :
        return  0

def tabuleiro(_X,_M) :
    [m,n] = _M.shape
    iv = np.arange(0,m)
    random.shuffle(iv)
    jv = np.arange(0,n)
    random.shuffle(jv) 
    jogada=0
    for i in iv  :
        for j in jv :
            if _X[i,j] == 1 :      
                jogada = jogada + 1                    
                _M[i,j] = _M[i,j] + 2
                # imprimir(_M,jogada) 
                if i > 0 :
                    _M[i-1,j] = inverter(_M[i-1,j])
                if i < m-1 :
                    _M[i+1,j] = inverter(_M[i+1,j])
                if j > 0 :
                    _M[i,j-1] = inverter(_M[i,j-1])
                if j < n-1 :                
                    _M[i,j+1] = inverter(_M[i,j+1])
                _M[i,j] = _M[i,j] - 2
                _M[i,j] = inverter(_M[i,j])          
                # imprimir(_M,jogada)   
 
    return _M   

main()

Número de jogadas:  15.0
Tabuleiro resolvido com sucesso


## Questão 2: Sudoku

### Modelo:

\begin{align*}
    \text{Minimizar }   & ijk x_{ijk}\\
    \text{sujeito a }   & \sum_{k} x_{ijk} = 1 \quad \forall  i,j  \hspace{20pt} \text{(Alocar apenas um número a cada casa)} \\
                        & \sum_{i} x_{ijk} = 1 \quad \forall  j,k  \hspace{20pt} \text{(Unicidade de cada número em cada coluna)} \\
                        & \sum_{j} x_{ijk} = 1 \quad \forall  i,k  \hspace{20pt} \text{(Unicidade de cada número em cada linha)} \\
                        & \sum_{p,p',q,q'} x_{(3p+q),(3p'+q'),k} = 1 \quad \forall  k;  \quad p,p',q,q' \in \{0,1,2\} \hspace{20pt} \text{(Unicidade de cada número em cada submatriz)} \\
                        & x_{ijk} = 1 \quad \forall i,j,k: \: \text{board}(i,j) = k \hspace{20pt} \text{(Fixar valores do tabuleiro inicial)}\\
                        & x_{ijk}\in \mathbb{B} \quad \forall  i,j,k  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
\end{align*}

onde os índices $(i,j,k)$ que indexam as variáveis são números interios no intervalo $[0,8]$, e as variáveis de decisão $x_{ijk} \in \mathbb{B}$ são não nulas $\iff$ $k$ for o número escolhido para a casa $(i,j)$ do tabuleiro.  Assumimos que o tabuleiro inicial é dado por uma matriz $\text{board}$ cujas casas vazias são preenchidas com zero.

Como o problema pode ser altamente degenerado (por se tratar de um problema de factibilidade), definimos um ordenamento arbitrário das soluções através da função objetivo.

In [59]:
def SolveSudoku(board:np.ndarray) -> np.ndarray:
    """ Solves a sudoku board by building and solving a linear program. 

    Args:
        board (np.ndarray): An array with the initial board values. Empty cells are assumed to be zero-valued.

    Returns:
        np.ndarray: A 3d array containing the optimal values of the model's binary decision variables.
    """

    boardSize = len(board[:,0])
    submatrixSize = 3
    digits = range(boardSize)
    submatrices = range(submatrixSize)

    # decision variables
    decisionVars = np.empty((boardSize, boardSize, boardSize), dtype=object)
    for i in digits:
        for j in digits:
            for k in digits:
                decisionVars[i,j,k] = cp.Variable(boolean=True, name=f"x_{i}{j}{k}")

    # constraints
    constraints = []
    # one single value per cell
    for i in digits:
        for j in digits:
            constraints += [np.sum(decisionVars[i,j,:]) == 1]

    # uniqueness in every column
    for j in digits:
        for k in digits:
            constraints += [np.sum(decisionVars[:,j,k]) == 1]
    
    # uniqueness in every row
    for i in digits:
        for k in digits:
            constraints += [np.sum(decisionVars[i,:,k]) == 1]

    # uniqueness in every sub-matrix
    for k in digits:
        for blockRow in submatrices:
            for blockCol in submatrices:
                constraint_expr = cp.sum([decisionVars[3*blockRow + i, 3*blockCol + j, k]
                                        for i in submatrices
                                        for j in submatrices])
                constraints.append(constraint_expr == 1)

    # initial board values 
    for i in digits:
        for j in digits:
            boardValue = board[i,j]
            if boardValue != 0:
                constraints.append(decisionVars[i, j, board[i,j] - 1] == 1)

    # building arbitrary objective function
    objectiveExpression = cp.sum([i*j*k*decisionVars[i,j,k] 
                                  for i in digits 
                                  for j in digits 
                                  for k in digits])
    objective = cp.Minimize(0)

    lp = cp.Problem(objective, constraints)
    lp.solve(solver="MOSEK", verbose=False)

    return decisionVars

def DecodeSolution(decisionVars:np.ndarray) -> np.ndarray:
    """ From a 3d array with binary variables representing a possible solution, build the corresponding 2d board.

    Args:
        decisionVars (np.ndarray): Binary decision variables representing a possible solution to a sudoku board.

    Returns:
        np.ndarray: Reconstructed sudoku board.
    """

    boardSize = len(decisionVars[0,:,:])
    digits = range(boardSize)

    solution = np.zeros((boardSize, boardSize), dtype=int)
    for i in digits:
        for j in digits:
            for k in digits:
                if decisionVars[i, j, k].value > 0.5:
                    solution[i, j] = k + 1
                    break
                
    return solution

def CheckSudoku(board):
    # Check if all rows contain digits 1-9 without duplicates
    for row in board:
        if not isValidGroup(row):
            return False

    # Check if all columns contain digits 1-9 without duplicates
    for col in range(9):
        column = [board[row][col] for row in range(9)]
        if not isValidGroup(column):
            return False

    # Check each 3x3 sub-box
    for box_row in range(3):
        for box_col in range(3):
            block = []
            for r in range(box_row * 3, box_row * 3 + 3):
                for c in range(box_col * 3, box_col * 3 + 3):
                    block.append(board[r][c])
            if not isValidGroup(block):
                return False

    return True

def isValidGroup(nums):
    # Check if the group contains digits 1-9 exactly once
    nums = [n for n in nums if n != 0]  # Ignore zeros if you want to allow incomplete boards
    return len(nums) == 9 and set(nums) == set(range(1, 10))

In [60]:
board = np.array([
    [0, 0, 0, 0, 0, 3, 0, 8, 5],
    [0, 0, 1, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 7, 0, 1, 3, 0, 0],
    [0, 0, 3, 0, 0, 0, 0, 1, 0],
    [9, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 5, 0, 0, 0, 0, 6, 0, 0],
    [0, 0, 7, 3, 0, 8, 0, 0, 0],
    [0, 0, 0, 0, 6, 0, 4, 0, 0],
    [3, 9, 0, 4, 0, 0, 0, 0, 0]
]
)
print("Initial Sudoku board: ")
print(board)
print("")
solution = SolveSudoku(board)

solution = DecodeSolution(solution)
print("Solved Sudoku board: ")
print(solution)
print("")
print("Is the board correctly filled: ", CheckSudoku(solution))

Initial Sudoku board: 
[[0 0 0 0 0 3 0 8 5]
 [0 0 1 0 2 0 0 0 0]
 [0 0 0 7 0 1 3 0 0]
 [0 0 3 0 0 0 0 1 0]
 [9 0 0 0 0 0 0 0 4]
 [0 5 0 0 0 0 6 0 0]
 [0 0 7 3 0 8 0 0 0]
 [0 0 0 0 6 0 4 0 0]
 [3 9 0 4 0 0 0 0 0]]

Solved Sudoku board: 
[[7 4 2 6 9 3 1 8 5]
 [6 3 1 8 2 5 7 4 9]
 [5 8 9 7 4 1 3 6 2]
 [2 6 3 5 7 4 9 1 8]
 [9 7 8 1 3 6 2 5 4]
 [1 5 4 9 8 2 6 7 3]
 [4 2 7 3 1 8 5 9 6]
 [8 1 5 2 6 9 4 3 7]
 [3 9 6 4 5 7 8 2 1]]

Is the board correctly filled:  True


## Questão 3: Senha

## Questão 4: 8 rainhas